# On import les bibliotheques et le dataframe

In [1]:
import numpy as np
import pandas as pd
import re
import unidecode

In [2]:
df = pd.read_csv('offres_emploi_concatene_raw.csv')

# On crée notre dictionnaire qui va nous servir pour convertir nos chiffre en str qui ont du sens

In [3]:
etude_num_2_str_dict  = {
    0 : "Aucune formation",
    1 : "4eme achevee",
    2 : "3eme achevee ou Brevet",
    3 : "CAP, BEP ou equivalent",
    4 : "Bac ou equivalent",
    5 : "Bac +2 ou equivalent",
    6 : "Bac +3 ou equivalent",
    7 : "Bac +4 ou equivalent",
    8 : "Bac +5 ou equivalent"
}

typeContrat_num_2_str_dict = {
    0 : 'CDI',
    1 : 'CDD',
    2 : 'MIS', 
    3 : 'DIN', 
    4 : 'CCE', 
    5 : 'LIB', 
    6 : 'FRA', 
    7 : 'SAI', 
    8 : 'REP', 
    9 : 'DDI', 
    10 : 'TTI'
}

tps_Pp_num_2_str_dict = {
    0 : "Temps partiel",
    1 : "Temps plein"
    }

In [4]:

def fill_formation(niveau):
    # Normalisation : suppression des accents et transformation en minuscules
    if pd.isna(niveau): return None
    niveau = str(niveau)
    niveau = unidecode.unidecode(niveau.lower())
    # Application des regex sur le texte normalisé
    if re.search(r"\baucune\b", niveau):  
        return 0
    elif re.search(r"\b4eme\b", niveau):  
        return 1
    elif re.search(r"\b(3[\s]*eme|brevet)\b", niveau):  
        return 2
    elif re.search(r"\b(cap|bep)\b", niveau):  
        return 3
    elif re.search(r"\b(bac|baccalaureat)\b(?!\s*\+)", niveau):  
        return 4
    elif re.search(r"\b(bac[\s]*\+[\s]*2|licence [\s]*2)\b", niveau):  
        return 5
    elif re.search(r"\b(bac[\s]*\+[\s]*3|licence|licenses)\b", niveau):  
        return 6
    elif re.search(r"\b(bac[\s]*\+[\s]*4|master[\s]*1)\b", niveau):  
        return 7
    elif re.search(r"\b(master|master[\s]*2|bac[\s]*\+[\s]*5)\b", niveau):  
        return 8
    return None

def fill_type_contrat(niveau):
    if pd.isna(niveau):
        return None
    
    elif niveau == 'CDI':
        return 0
    elif niveau == 'CDD':
        return 1
    elif niveau == 'MIS':
        return 2
    elif niveau == 'DIN':
        return 3
    elif niveau == 'CCE':
        return 4
    elif niveau == 'LIB':
        return 5
    elif niveau == 'FRA':
        return 6
    elif niveau == 'SAI':
        return 7
    elif niveau == 'REP':
        return 8
    elif niveau == 'DDI':
        return 9
    elif niveau == 'TTI':
        return 10
    else:
        return None  #Si le type de contrat ne correspond pas
    
def fill_departement(row):
    # Si lieuTravail.codePostal est rempli
    if pd.notna(row['lieuTravail.codePostal']):
        try:
            return int(str(row['lieuTravail.codePostal'])[:2])  # Prend les 2 premiers caractères
        except ValueError:
            return None  # Si conversion en int échoue
    
    # Sinon, on vérifie lieuTravail.libelle
    if pd.notna(row['lieuTravail.libelle']):
        parts = row['lieuTravail.libelle'].split(' ')[0]  # Prend la première partie du libellé
        if parts.isdigit():  # Vérifie si la partie est un nombre
            return int(parts[:2])  # Retourne les 2 premiers caractères comme un entier
    
    return None

def fill_experience(niveau):
    if pd.isna(niveau):
        return None
    
    niveau = str(niveau)
    niveau = unidecode.unidecode(niveau.lower())
    
    #Débutant Ok
    if re.search(r"\bdebutant\b", niveau):
        return 0
    
    #Temps requis en année
    elif re.search(r"\b(\d+)\s*(an|ans)\b", niveau):
        match_annees = re.search(r"\b(\d+)\s*(an|ans)\b", niveau)
        return int(match_annees.group(1)) * 12
    
    #Temps requis en mois
    elif re.search(r"\b(\d+)\s*mois\b", niveau):
        match_mois = re.search(r"\b(\d+)\s*mois\b", niveau)
        return int(match_mois.group(1))
        
    else:
        return None
    
def fill_romeCode(niveau):
    # Vérifie si la valeur est NaN
    if pd.isna(niveau):
        return None
    
    # Convertit la valeur en chaîne de caractères
    niveau = str(niveau)
    
    # Vérifie si la longueur de la chaîne est suffisante
    if len(niveau) < 2:
        return None  # Retourne None si la chaîne est trop courte
    
    # Retourne les deux premiers caractères
    return niveau[:3]

def fill_tps_Pp(niveau):
    if pd.isna(niveau):
        return None
        
    elif niveau == 'Temps plein':
        return 1
    elif niveau == 'Temps partiel':
        return 0

    return None

def fill_salaire(salaire):
    if pd.isna(salaire):  # Gérer les valeurs manquantes
        return None

    salaire = str(salaire).replace(",", ".")  # Remplacer les virgules par des points
    salaire = re.sub(r'(\d+(\.\d+)?\s?mois)', '', salaire, flags=re.IGNORECASE)  # Supprimer '12 mois', '13 mois', etc.

    # Déterminer la catégorie : Horaire, Mensuel, Annuel
    if "horaire" in salaire.lower():
        category = "horaire"
    elif "mensuel" in salaire.lower():
        category = "mensuel"
    elif "annuel" in salaire.lower():
        category = "annuel"
    else:
        category = "unknown"  # Catégorie inconnue, on utilise la devinette

    # Extraire les chiffres du salaire
    numbers = [float(num) for num in re.findall(r'\d+(?:\.\d+)?', salaire)]

    if len(numbers) == 0:  # Aucune valeur numérique trouvée
        return None
    elif len(numbers) >= 3:  # Si trois chiffres trouvés, ne rien faire
        return None
    elif len(numbers) == 2:  # Deux chiffres trouvés, vérifier l'écart
        lower, higher = sorted(numbers)
        if higher / lower > 3:  # Si l'écart dépasse 12 fois, retourner None
            return None
        salary_value = sum(numbers) / 2  # Sinon, prendre la moyenne
    else:  # Un seul chiffre trouvé
        salary_value = numbers[0]

    # Logique de devinette si la catégorie est inconnue
    if category == "unknown":
        if salary_value < 250:  # En dessous de 250, c'est horaire
            category = "horaire"
        elif 250 <= salary_value <= 5000:  # Entre 250 et 5000, c'est mensuel
            category = "mensuel"
        elif salary_value > 5000:  # Au-dessus de 5000, c'est annuel
            category = "annuel"

    # Conversion en salaire annuel
    if category == "horaire":
        return round(salary_value * 35 * 52)  # 35 heures/semaine * 52 semaines
    elif category == "mensuel":
        return round(salary_value * 12)  # 12 mois
    elif category == "annuel":
        return round(salary_value)  # Déjà en annuel
    
    return None

def fill_secteur_activite(row):
    # Si secteurActivite est rempli
    if pd.notna(row['secteurActivite']):
        return row['secteurActivite']
    
    # Sinon, on vérifie codeNAF
    if pd.notna(row['codeNAF']):
        code_naf = str(row['codeNAF'])
        if code_naf[:2].isdigit():  # Vérifie si les 2 premiers caractères sont des chiffres
            return int(code_naf[:2])  # Retourne les 2 premiers caractères comme un entier
    
    return None
    


In [5]:
df['departement'] = df.apply(fill_departement, axis=1)
df['salaire_annuel'] = df['salaire.libelle'].apply(fill_salaire)
df['etude_requise'] = df['formations.niveauLibelle'].apply(fill_formation)
df['experience_requise'] = df['experienceLibelle'].apply(fill_experience)
df["contrat"] = df['typeContrat'].apply(fill_type_contrat)
df['romeCode_extract'] = df["romeCode"].apply(fill_romeCode)
df['tps_Pp'] = df['dureeTravailLibelleConverti'].apply(fill_tps_Pp)
df['secteur_activite'] = df.apply(fill_secteur_activite, axis=1)

In [6]:
print(df.columns)

Index(['intitule', 'lieuTravail.libelle', 'lieuTravail.codePostal', 'romeCode',
       'secteurActivite', 'codeNAF', 'typeContrat', 'salaire.libelle',
       'dureeTravailLibelleConverti', 'experienceLibelle',
       'formations.niveauLibelle', 'qualificationCode', 'qualificationLibelle',
       'description', 'source', 'departement', 'salaire_annuel',
       'etude_requise', 'experience_requise', 'contrat', 'romeCode_extract',
       'tps_Pp', 'secteur_activite'],
      dtype='object')


# On va ajuster manuellement les salaires trop haut et trop bas a cause des erreurs dans les offres...

In [7]:
heures_par_semaine = 35
semaines_par_an = 52
heures_annuelles = heures_par_semaine * semaines_par_an
heures_mensuelles = heures_annuelles / 12

def ajuster_salaire(salaire):
    # Correction pour les salaires trop élevés
    if salaire > heures_annuelles * 10000:  # Salaire annuel qui a été mis en horaire
        return salaire / heures_annuelles
    elif heures_annuelles * 1000 < salaire <= heures_annuelles * 10000:  # Salaire mensuel transformé en horaire
        return salaire / heures_mensuelles
    elif 220000 < salaire <= heures_annuelles * 1000:  # Salaire annuel compté comme mensuel
        return salaire / 12
    
    # Correction pour les salaires trop faibles
    elif salaire < 110:  # Salaire horaire transformé en annuel
        return salaire * heures_annuelles
    elif 110 <= salaire <= 1000:  # Salaire horaire transformé en mensuel
        return salaire * heures_mensuelles
    elif 1000 <= salaire <= 3000:  # Salaire mensuel transformé en annuel
        return salaire * 12

    # Pas de transformation nécessaire
    return salaire

df['salaire_annuel'] = df['salaire_annuel'].apply(ajuster_salaire)

# On va lire la description pour compléter :/ 

## Pour le salaire

In [8]:
import re
import unidecode

def add_salary(description):
    # Remplacement des symboles problématiques
    description = description.replace('€', 'EUR')
    description = description.replace(',', '.')
    description = description.replace('¿', ' ')  # Remplace le symbole problématique
    description = description.replace('*', '  ') # Remplace l'astérisque par deux espaces pour faire la coupure
    description = description.replace('/', 'par') # remplacer les "/" par "par "
    # Remplacer les espaces dans les nombres par "rien"
    description = re.sub(r'(?<=\d) (?=\d)', '', description)
    description = re.sub(r'(?<=\d)[\s\u202f\xa0](?=\d)', '', description)
    # Normaliser le texte
    description = unidecode.unidecode(description.lower())
    start_pos = 0  # Position de départ pour la recherche
    salary = None
    category = None
    
    while start_pos < len(description):
        # Identifier le mot 'salaire' ou 'rémunération'
        match = re.search(r'\b(salaire|remuneration)\b', description[start_pos:])
        if not match:
            break  # Pas de mention de salaire/rémunération restante dans le texte
        
        # Position de fin du mot clé trouvé
        match_end = start_pos + match.end()
        
        remaining_text = description[match_end:match_end + 110]  # On cherche plus large pour vérifier
        # Trouver le premier "." non suivi d'un chiffre
        valid_stop_dot = re.search(r'\.(?!\d)', remaining_text)
        # Trouver le premier double espace "  "
        valid_stop_space = re.search(r'  ', remaining_text)

        stop_pos_dot = valid_stop_dot.start() if valid_stop_dot else len(remaining_text)
        stop_pos_space = valid_stop_space.start() if valid_stop_space else len(remaining_text)

        # Prendre le minimum entre 110, le point valide, et le double espace
        stop_pos = min(110, stop_pos_dot, stop_pos_space)
        context = remaining_text[:stop_pos]
                
        # Parcourir les mots du contexte pour détecter le premier montant et la première catégorie
        words = context.split()
        for i in range(len(words) - 1):
            # Vérification des catégories avec deux mots consécutifs
            two_words = f"{words[i]} {words[i+1]}"
            if re.search(r'\b(par[\s]*an|par[\s]*ans|annuel|annuels|annuelle|annuelles)\b', two_words):
                category = "annuel"
            elif re.search(r'\b(mensuel|mensuels|mensuelle|mensuelles|par[\s]*mois)\b', two_words):
                category = "mensuel"
            elif re.search(r'\b(horaire|horaires|par[\s]*heure|par[\s]*heures)\b', two_words):
                category = "horaire"
            
            # Chercher le montant avec EUR ou euro
            if salary is None:
                amount_match = re.match(r'(\d+(?:\.\d+)?)(?:\s*(k|eur|euro|euros|keur|keuros|keuro)\b)', two_words)
                if amount_match:
                    salary = float(amount_match.group(1))
                    if amount_match.group(2) and amount_match.group(2).lower() in ('k', 'keur', 'keuros', 'keuro'):
                        if salary < 500:
                            salary *= 1000 
                            category = "annuel"
                            break
                        else:
                            category = "annuel"
                            break
            
            # Si on a trouvé le montant et la catégorie, on peut arrêter
            if salary is not None and category is not None:
                break
        
        # Si montant et catégorie trouvés, et valeur crédible on arrête. 
        if salary is not None and category is not None:
            if category == 'mensuel' and salary * 12 > 5000:
                break
            if category == 'horaire' and salary * 35 * 52 > 9*35*52:
                break
            if salary > 5000:
                break
        
        # Déplacer la position de départ pour continuer la recherche
        salary = None
        category = None
        start_pos = match_end
    
    if salary is None or category is None:
        return None  # Montant ou catégorie non trouvé
    
    # Convertir le salaire en base annuelle si nécessaire
    if category == "mensuel":
        salary *= 12
    elif category == "horaire":
        salary *= 35 * 52
    
    return round(salary)  # Arrondir le salaire

def add_salary_df(df):
    # Créer un masque pour les lignes où 'salaire_annuel' est NaN
    mask = df["salaire_annuel"].isna()
    
    # Appliquer la fonction add_salary uniquement sur les lignes correspondantes
    df.loc[mask, "salaire_annuel"] = df.loc[mask, "description"].apply(add_salary)
    
    # Retourner le DataFrame mis à jour
    return df


In [9]:

percent_nan_before = df['salaire_annuel'].isna().mean() * 100
df = add_salary_df(df)
percent_nan_after = df['salaire_annuel'].isna().mean() * 100

print(f"Pourcentage de NaN avant : {percent_nan_before:.2f}%")
print(f"Pourcentage de NaN après : {percent_nan_after:.2f}%")


Pourcentage de NaN avant : 56.51%
Pourcentage de NaN après : 52.99%


## Pour temps plein ou partiel

In [10]:
def add_tps_Pp(description):
    description = unidecode.unidecode(description.lower())
    # Regex pour matcher toutes les variantes
    if re.search(r"temps\s*pleins?", description):
        return 1
    elif re.search(r"temps\s*partiels?", description):
        return 0
    return np.nan


def add_tps_Pp_df(df):
    mask = df["tps_Pp"].isna()
    df.loc[mask, "tps_Pp"] = df.loc[mask, "description"].apply(add_tps_Pp)
    return df

In [11]:
percent_nan_before = df['tps_Pp'].isna().mean() * 100
df = add_tps_Pp_df(df)

percent_nan_after = df['tps_Pp'].isna().mean() * 100

print(f"Pourcentage de NaN avant : {percent_nan_before:.2f}%")
print(f"Pourcentage de NaN après : {percent_nan_after:.2f}%")


Pourcentage de NaN avant : 47.56%
Pourcentage de NaN après : 42.09%


## Pour le niveau d'éducation requis

In [12]:
def add_education(description):
    description = unidecode.unidecode(description.lower())
    if re.search(r"\baucune\b", description):  
        return 0
    elif re.search(r"\b4eme\b", description):  
        return 1
    elif re.search(r"\b(3[\s]*eme|brevet)\b", description):  
        return 2
    elif re.search(r"\b(cap|bep)\b", description):  
        return 3
    elif re.search(r"\b(bac|baccalaureat)\b(?!\s*\+)", description):  
        return 4
    elif re.search(r"\b(bac[\s]*\+[\s]*2|licence [\s]*2)\b", description):  
        return 5
    elif re.search(r"\b(bac[\s]*\+[\s]*3|licence|licenses)\b", description):  
        return 6
    elif re.search(r"\b(bac[\s]*\+[\s]*4|master[\s]*1)\b", description):  
        return 7
    elif re.search(r"\b(master|master[\s]*2|bac[\s]*\+[\s]*5)\b", description):  
        return 8
    return None

def add_education_df(df):
    mask = df["etude_requise"].isna()
    df.loc[mask, "etude_requise"] = df.loc[mask, "description"].apply(add_education)
    return df
    

In [13]:
# Pourcentage de NaN avant
percent_nan_before = df['etude_requise'].isna().mean() * 100

# Appliquer la fonction
df = add_education_df(df)

# Pourcentage de NaN après
percent_nan_after = df['etude_requise'].isna().mean() * 100

print(f"Pourcentage de NaN avant : {percent_nan_before:.2f}%")
print(f"Pourcentage de NaN après : {percent_nan_after:.2f}%")


Pourcentage de NaN avant : 76.14%
Pourcentage de NaN après : 52.79%


## Pour le temps d'expérience nécessaire

In [14]:
def add_experience(description):
    # Normalisation de la description
    description = unidecode.unidecode(description.lower())

    start_pos = 0  # Position de départ pour la recherche
    experience = None
    while start_pos < len(description):
        # Identifier le mot 'experience'
        match = re.search(r'\bexperience\b', description[start_pos:])
        if not match:
            break  # Pas de mention d'expérience restante dans le texte

        # Position de fin du mot clé trouvé
        match_end = start_pos + match.end()

        # Prendre le contexte autour du mot clé
        context_start = max(0, match.start() - 15)  # S'assurer que le début est >= 0
        context_end = match_end + 70
        context = description[context_start:context_end]
        # Vérifier si "premiere experience" est mentionné
        if "premiere experience" in context:
            experience = 6  # Retourner 6 mois
            return experience

        # Vérifier la présence d'un nombre suivi de "ans" ou "an"
        match_years = re.search(r'(\d+)\s*(ans|an)', context)
        if match_years:
            experience = int(match_years.group(1)) * 12  # Convertir en mois
            return experience

        # Déplacer la position de départ pour continuer la recherche
        start_pos = match_end

    return np.nan # Retourne l'expérience trouvée ou None si non trouvé

def add_experience_df(df):
    mask = df["experience_requise"].isna()
    df.loc[mask, "experience_requise"] = df.loc[mask, "description"].apply(add_experience)
    return df


In [15]:
percent_nan_before = df['experience_requise'].isna().mean() * 100

df = add_experience_df(df)

percent_nan_after = df['experience_requise'].isna().mean() * 100

print(f"Pourcentage de NaN avant : {percent_nan_before:.2f}%")
print(f"Pourcentage de NaN après : {percent_nan_after:.2f}%")

Pourcentage de NaN avant : 9.65%
Pourcentage de NaN après : 4.70%


# On va enfin enlever arbitrairement les lignes où le salaire est supérieur a 300 000. trop outlier, et aussi les salaire inferieur a 

In [16]:
df = df[(df['salaire_annuel'].isna()) | (df['salaire_annuel'].between(4000, 300000))]

In [20]:
df.head()

,intitule,lieuTravail.libelle,lieuTravail.codePostal,romeCode,secteurActivite,codeNAF,typeContrat,salaire.libelle,dureeTravailLibelleConverti,experienceLibelle,...,description,source,departement,salaire_annuel,etude_requise,experience_requise,contrat,romeCode_extract,tps_Pp,secteur_activite
0,Administrateur systèmes et réseaux informatiqu...,41 - Blois,41000.0,M1801,78.0,78.20Z,MIS,Annuel de 38000.0 Euros sur 12.0 mois,Temps plein,2 An(s),...,"Rattaché au responsable de service, vous admin...",Informatique,41.0,38000.0,5.0,24.0,2,M18,1,78.0
1,Technicien informatique F/H,69 - Limonest,69760.0,H1108,78.0,78.20Z,MIS,Annuel de 23475.0 Euros à 25000.0 Euros sur 12...,Temps plein,1 An(s),...,Synergie Lyon ouest recrute un(e) technicien(n...,Informatique,69.0,24238.0,NaN,12.0,2,H11,1,78.0
2,Ingénieur de production/exploitation informati...,41 - Blois,41000.0,M1810,78.0,78.20Z,MIS,Annuel de 35000.0 Euros sur 12.0 mois,Temps plein,5 An(s),...,"Rattaché au responsable d'équipe, vous intégre...",Informatique,41.0,35000.0,5.0,60.0,2,M18,1,78.0
3,Technicien informatique (H/F),MONACO,NaN,I1404,66.0,66.22Z,CDI,Annuel de 27000.0 Euros sur 12.0 mois,Temps plein,Débutant accepté,...,CDI TEMPS PLEIN - prise de poste dès que possi...,Informatique,NaN,27000.0,5.0,0.0,0,I14,1,66.0
4,Chef de projet réseau informatique (H/F),74 - ANNEMASSE,74100.0,M1803,78.0,78.20Z,CDI,Annuel de 45000.0 Euros sur 12.0 mois,Temps plein,11 An(s),...,Vous souhaitez rejoindre une entreprise en ple...,Informatique,74.0,45000.0,NaN,132.0,0,M18,1,78.0


In [19]:
df.sort_values(by='salaire_annuel', ascending=False)[0:20]

,intitule,lieuTravail.libelle,lieuTravail.codePostal,romeCode,secteurActivite,codeNAF,typeContrat,salaire.libelle,dureeTravailLibelleConverti,experienceLibelle,...,description,source,departement,salaire_annuel,etude_requise,experience_requise,contrat,romeCode_extract,tps_Pp,secteur_activite
10917,Directeur / Directrice des Ressources Humaines...,91 - LES ULIS,91940.0,M1503,NaN,NaN,CDI,"Annuel de 90,00 Euros à 110,00 Euros",NaN,Expérience exigée de 3 An(s),...,Description du poste : Notre client : Société ...,Ressources_Humaines,91.0,182000.0,8.0,36.0,0,M15,NaN,NaN
13260,Directeur Administratif & Financier H/F - Algé...,Algérie,NaN,M1205,25.0,25.62B,CDI,"Annuel de 142838,00 Euros à 209496,00 Euros",NaN,Expérience exigée de 1 An(s),...,A la tête du département financier de cette Jo...,Finance,NaN,176167.0,NaN,12.0,0,M12,NaN,25.0
19098,Esthéticien / Conseiller beauté (H/F),37 - TOURS,37000.0,D1208,96.0,96.02B,CDD,Mensuel de 13010.16 Euros sur 12.0 mois,Temps partiel,24 Mois,...,"Nous sommes une entreprise prestigieuse, spéci...",Conseil,37.0,156122.0,3.0,24.0,1,D12,"(0,)",96.0
37780,Responsable environnement-hygiène-sécurité en ...,49 - CHOLET,49300.0,H1304,NaN,NaN,CDI,"Mensuel de 11760,00 Euros",NaN,Débutant accepté,...,"Description du poste : La convivialité, l'auto...",Environnement,49.0,141120.0,NaN,0.0,0,H13,1.0,NaN
10168,Directeur(trice) des Ressources Humaines H/F,France,NaN,M1503,45.0,45.11Z,CDI,"Annuel de 120000,00 Euros à 150000,00 Euros",NaN,Expérience exigée de 1 An(s),...,Missions : * Vous déployez la politique RH d...,Ressources_Humaines,NaN,135000.0,NaN,12.0,0,M15,NaN,45.0
12910,Directeur administratif et financier / Directr...,75 - PARIS 09,75009.0,M1205,NaN,NaN,CDI,"Annuel de 120000,00 Euros à 150000,00 Euros",NaN,Expérience exigée de 6 An(s),...,Description du poste : Nous recrutons pour l'u...,Finance,75.0,135000.0,8.0,72.0,0,M12,NaN,NaN
30443,Chef du service d'information et de communicat...,988 - Nouvelle Calédonie,NaN,E1109,NaN,NaN,CDI,"Mensuel de 66,00 Euros à 78,00 Euros",NaN,Expérience exigée de 5 An(s),...,Chef de Service Monétique et Nouveaux Moyens d...,Media,98.0,131040.0,6.0,60.0,0,E11,NaN,NaN
10770,Directeur / Directrice des Ressources Humaines...,75 - PARIS 09,75009.0,M1503,NaN,NaN,CDI,"Annuel de 110000,00 Euros à 150000,00 Euros",NaN,Expérience exigée de 3 An(s),...,Description du poste : Développement de la str...,Ressources_Humaines,75.0,130000.0,NaN,36.0,0,M15,NaN,NaN
14535,Directeur administratif et financier / Directr...,75 - PARIS 09,75009.0,M1205,NaN,NaN,CDI,"Annuel de 110000,00 Euros à 150000,00 Euros",NaN,Expérience exigée de 3 An(s),...,Description du poste : -Élaboration de la stra...,Finance,75.0,130000.0,NaN,36.0,0,M12,NaN,NaN
29991,Directeur de la Sécurité des Systèmes d'Inform...,France,NaN,M1812,52.0,52.29B,CDI,"Annuel de 110000,00 Euros à 140000,00 Euros",NaN,Expérience exigée de 1 An(s),...,En tant que Directeur de la Sécurité des Systè...,Media,NaN,125000.0,NaN,12.0,0,M18,NaN,52.0
